In [4]:
import re

j = 0

notyears = ['Argonautica','Dicaearchus','Editions','Epigram','Eutrop','Gennadius','Hephaestion','Liberalis','Lycophron','Petronius','Theogony','appended','chapter','epigrams','wrote']
notyears.append('Apollo')
notyears.append('Aphrodite')
notyears.append('Aristophanes')
notyears.append('Artemis')
notyears.append('Avienus')
notyears.append('Chius')
notyears.append('Demeter')
notyears.append('Festus')
notyears.append('Jornandes')
notyears.append('Obsequens')
notyears.append('Orelli')
notyears.append('Orpheus')
notyears.append('Periegetes')
notyears.append('Phrygius')
notyears.append('Scymnus')
notyears.append('Thesmophoriazusae')

rnums = {}
rnums['i'] = 1
rnums['ii'] = 2
rnums['iii'] = 3
rnums['iv'] = 4
rnums['v'] = 5
rnums['vi'] = 6
rnums['vii'] = 7
rnums['viii'] = 8
rnums['ix'] = 9
rnums['x'] = 10
rnums['xi'] = 11
rnums['xii'] = 12
rnums['xiii'] = 13
rnums['xiv'] = 14
rnums['xv'] = 15
rnums['xvi'] = 16
rnums['xvii'] = 17
rnums['xviii'] = 18
rnums['xix'] = 19
rnums['xx'] = 20
rnums['xxi'] = 21
rnums['xxii'] = 22
rnums['xxiii'] = 23
rnums['xxiv'] = 24
rnums['xxv'] = 25
rnums['xxvi'] = 26
rnums['xxvii'] = 27
rnums['xxviii'] = 28
rnums['xxix'] = 29
rnums['xxx'] = 30
rnums['xxxi'] = 31
rnums['xxxii'] = 32
rnums['xxxiii'] = 33
rnums['xxxiv'] = 34
rnums['xxxv'] = 35
rnums['xxxvi'] = 36
rnums['xxxvii'] = 37
rnums['xxxviii'] = 38
rnums['xxxix'] = 39
rnums['xl'] = 40
rnums['xli'] = 41
rnums['xlii'] = 42
rnums['xliii'] = 43
rnums['xliv'] = 44
rnums['xlv'] = 45
rnums['xlvi'] = 46
rnums['xlvii'] = 47
rnums['xlviii'] = 48
rnums['xlix'] = 49
rnums['l'] = 50


countwords = []
f2 = open('../002/count-words.txt')
for l in f2:
    l = re.sub('\n','',l)
    countwords.append(l)
f2.close()

def fract2dec(s):
    s = re.sub(',','',s)
    m = re.search('([0-9]+)\s+([0-9]+)/([0-9]+)',s)
    if(m):
        basen = int(m[1])
        numerator = int(m[2])
        denom = int(m[3])
        rval = basen + numerator/denom
        print('fractres',s,rval)
        s = str(rval)
        
    return(s)


f = open("viaf88890045.003.perseus-eng1.xml")
outf = open('smithbio-work1.xml','w')

entrylens = {}

idlist = {}
inheader = 1
needspace = 0
curvolume = ''
curentryid = ''
firstpara = 0
baseid = ''
for l in f:
    if(inheader):
        if(re.search('</teiHeader>',l)):
            inheader = 0
        print(l,end='',file=outf)
        continue
        
    m = re.search('<milestone unit="volume" n="([123])"/>',l)
    if(m):
        curvolume = m[1]
        
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        firstpara = 1
        curentryid = m[1]
        baseid = re.sub('-[0-9a-z]+$','',curentryid)
        if( baseid in idlist):
            idlist[baseid] = idlist[baseid] + 1
        else:
            idlist[baseid] = 1
        newid = baseid + '-' + str(idlist[baseid])
        if(not newid == curentryid):
            #print('changeid',curentryid,newid)
            l = re.sub('id="'+curentryid+'"','id="'+newid+'"',l)
            curentryid = newid

        entrylens[curentryid] = 0
    
    
    if( re.search('<p>',l)):
        m = re.search('<p>([0-9]+)\.',l)
        if(m and firstpara and baseid and not str(idlist[baseid]) == m[1]):
            print('badn',curentryid,baseid,idlist[baseid],l)
        firstpara = 0
    if(curentryid in entrylens):
        entrylens[curentryid] = entrylens[curentryid] + len(re.sub('<[^<]+>','',l).split())


    l = re.sub('\s*\n',' ',l)
    l = re.sub('^\s+','',l)
    l = re.sub('([,;])</hi>','</hi>\g<1>',l)
    if(1 and re.search('^\s*(<div|<by|<p>|</div>)',l)):
        l = re.sub('^\s*(<item|<div|<by|<p>|</div>)','\n\n\g<1>',l)
    if(re.search('^\s*(<head|<milestone)',l)):
        l = re.sub('^\s*(<head|<milestone)','\n\g<1>',l)
    print(l,end='',file=outf)

f.close()
outf.close()

singnameindex = {}
for foo in entrylens:
    workid = re.sub('-bio.*','',foo)
    #headtext = re.sub('<[^>]+>',' ',entryheads[foo].lower())
    #headtext = re.sub('[\.;:!,]+',' ',headtext)
    #for tmpkey in headtext.split():
    for tmpkey in workid.split('-'):
        if(len(tmpkey)<3):
            continue
        if(not tmpkey in singnameindex):
            singnameindex[tmpkey] = {}
        singnameindex[tmpkey][foo] = entrylens[foo]

f = open('smithbio-work1.xml')
outf = open('smithbio-work.xml','w')

entryheads = {}
entrydates = {}
entrytitles = {}
sentences = 0
inheader = 1
curpage = ''
curvolume = ''
curentryid = ''

shortnames = ['Cato','Hera','Jew','Jews','Livy','Milo','Num','Otho','Ovid','Piso','Pius','Way','Ways','West','Zeno','Zeus']
shortpat = '('+ '|'.join(shortnames) + ')'

newdates = 0
newdates1 = 0
newdates2 = 0
newdates3 = 0

centval = {}
centval["15th B.C."] = "-1500-1401"
centval["twelfth B.C."] = "-1200-1101"
centval["tenth B.C."] = "-1000-901"
centval["eleventh B.C."] = "-1100-1001"
centval["ninth B.C."] = "-900-801"
centval["8th B.C."] = "-800-701"
centval["eighth B.C."] = "-800-701"
centval["seventh B.C."] = "-700-601"
centval["7th B.C."] = "-700-601"
centval["sixth B.C."] = "-600-501"
centval["6th B.C."] = "-600-501"
centval["5th B.C."] = "-500-401"
centval["fifth B.C."] = "-500-401"
centval["fourth B.C."] = "-100-101"
centval["4th B.C."] = "-400-301"
centval["3rd B.C."] = "-300-201"
centval["third B.C."] = "-300-201"
centval["second B.C."] = "-200-101"
centval["1st B.C."] = "-100-001"
centval["first B.C."] = "-100-001"
centval["a B.C."] = "-100-001"
centval["the B.C."] = "-100-001"
centval["last B.C."] = "-100-001"
centval["first A.D."] = "001-100"
centval["2nd A.D."] = "101-200"
centval["second A.D."] = "101-200"
centval["3rd A.D."] = "201-300"
centval["third A.D."] = "201-300"
centval["4th A.D."] = "301-400"
centval["fourth A.D."] = "301-400"
centval["fifth A.D."] = "401-500"
centval["5th A.D."] = "401-500"
centval["sixth A.D."] = "501-600"
centval["6th A.D."] = "501-600"
centval["seventh A.D."] = "601-700"
centval["7th A.D."] = "601-700"
centval["eighth A.D."] = "701-800"
centval["8th A.D."] = "701-800"
centval["ninth A.D."] = "801-900"

centval["9th A.D."] = "801-900"
centval["10th A.D."] = "901-1000"
centval["tenth A.D."] = "901-1000"
centval["eleventh A.D."] = "1001-1100"
centval["12th A.D."] = "1101-1200"
centval["twelfth A.D."] = "1101-1200"
centval["thirteenth A.D."] = "1201-1300"
centval["13th A.D."] = "1201-1300"
centval["fourteenth A.D."] = "1301-1400"
centval["14th A.D."] = "1301-1400"
centval["fifteenth A.D."] = "1401-1500"
centval["15th A.D."] = "1401-1500"
centval["sixteenth A.D."] = "1501-1600"
centval["16th A.D."] = "1501-1600"

goodxrefs = 0
ambxrefs = 0
xrefs = 0
for l in f:
    l = re.sub('<hi rend="ital">([A-Z][a-z]+\. [^<]+)</hi>','<title>\g<1></title>',l)
    m = re.search('<milestone unit="volume" n="([0-9]+)"',l)
    l,hits = re.subn('(\s+|\()(A\.\s*D\.)\s*([0-9]+)','\g<1><date when-custom="\g<3>">\g<2> \g<3></date>',l)
    newdates1 = newdates1 + hits
    l,hits = re.subn('(\s+|\()(B\.\s*C\.)\s*([0-9]+)','\g<1><date when-custom="-\g<3>">\g<2> \g<3></date>',l)
    newdates1 = newdates1 + hits
    l,hits = re.subn('([0-9]+)([,\s]*)(B\.\s*C\.)','<date when-custom="-\g<1>">\g<1>\g<2>\g<3></date>',l)
    newdates1 = newdates1 + hits
    l,hits = re.subn('([0-9]+)([,\s]*)(A\.\s*D\.)','<date when-custom="\g<1>">\g<1>\g<2>\g<3></date>',l)
    newdates1 = newdates1 + hits
    if(m):
        curvolume = m[1]
        
    m = re.search('<pb n="([0-9ivx]+)"',l)
    if(m):
        curpage = m[1]
        if( curvolume == '1'):
        
            if(re.search('^[0-9]+$',curpage)):
                l = re.sub('(<pb[^>/]+)','\g<1> source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044099890501%25seq=' + str(18+int(curpage))+'"',l)
            elif(curpage in rnums):
                l = re.sub('(<pb[^>/]+)','\g<1> source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044099890501%25seq=' + str(6+int(rnums[curpage]))+'"',l)
    
        if( curvolume == '2'):
            l = re.sub('(<pb[^>/]+)','\g<1> source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044011634888%25seq=' + str(26+int(curpage))+'"',l)
        if( curvolume == '3'):
            l = re.sub('(<pb[^>/]+)','\g<1> source="https://babel.hathitrust.org/cgi/pt?id=nnc1.cr00106801%25seq=' + str(8+int(curpage))+'"',l)
        
    if(re.search('<head',l)):
        curhead = re.sub('<[^>]+>','',l)
        curhead = re.sub('\n','',curhead)
        if( not curentryid in entryheads):
            entryheads[curentryid] = curhead
        
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        curentryid = m[1]
        #entrylens[curentryid] = 0
        entrydates[curentryid] = []
        entrytitles[curentryid] = []
        
    if(inheader):
        if(re.search('</teiHeader>',l)):
            inheader = 0
        print(l,end='',file=outf)
        continue

    i = 0 
    matchs = '<date when-custom="([0-9]+|\-[0-9]+)">([^<]+</date>)(\s+and\s+|\s+or\s+|\s+to\s+|\s+till\s+|,\s+)([0-9]+|\-[0-9]+)'
    m = re.search(matchs,l)
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        newdates = newdates + 1
        date1 = m[1]
        linkword = m[3]
        date2 = m[4]
        if(re.search('^\-',date1)):
            date2 = '-' + date2
            #print('date2 now',date1,date2,l)
        i = i + 1
        if(i> 100):
            print('outofcontrol1',l)
            break
        if(0 and int(date2) < int(date1)):
            print('newdate',curentryid,curvolume,curpage,date1,linkword,date2,l,end='')
        l = re.sub(matchs,'<date when-custom="\g<1>">\g<2>\g<3><date when-custom="\g<4>">\g<4></date>',l,1)


    l,hits = re.subn('(</date>)\s*-\s*([0-9]+)','\g<1>-<date when-custom="\g<2>">\g<2></date>',l)
    newdates2 = newdates2 + hits
    
    workl = l
    matchs = '<date when-custom="([0-9\-]+)">([^<]+)</date>-<date when-custom="([0-9\-]+)">'
    while(re.search(matchs,workl)):
        m = re.search(matchs,workl)
        date1 = m[1]
        date2 = m[3]
        if(not len(date1) == len(date2) and int(date1) > int(date2)):
            datemask = re.sub('(.)','.',date2) + '$'
            half1 = re.sub(datemask,'',date1)
            newdate2 = half1 + date2
            if(1 or int(newdate2) < int(date1)):
                print('unmatched',date1,date2,newdate2)
            l = re.sub('when-custom="' + date2+ '"','when-custom="' + newdate2 + '"',l)
        workl = re.sub(matchs,'',workl,1)


        
    
    workl = l
    while(re.search('<title[^>]*>([^<]+)',workl)):
        m = re.search('<title[^>]*>([^<]+)',workl)
        if(curentryid in entrytitles):
            entrytitles[curentryid].append(m[1])
        workl = re.sub('<title[^>]*>([^<]+)','',workl,1)
        
    workl = l
        
    l = re.sub('(<hi rend="smallcaps">[^<]+)</hi>\s+<hi rend="smallcaps">','\g<1> ',l)
    if(re.search('<p>',l)):
        l = re.sub('([A-Z][a-z][a-z]\.)','<abbr>\g<1></abbr>',l)
        l = re.sub('<abbr>'+shortpat+'\.</abbr>','\g<1>',l)
        while(re.search('(="[^"]*|<bibl[^>]*>[^<]*|<title[^>]*>[^<]*)<[/]*abbr>',l)):
            l = re.sub('(="[^"]*|<bibl[^>]*>[^<]*|<title[^>]*>[^<]*)<[/]*abbr>','\g<1>',l)
        l,hits = re.subn('([a-z][a-z]+[\.!?"])\s+([A-Z][a-z]*\s+[A-Za-z]+[,\s]|[A-Z][a-z]*\s+<[a-zA-Z]+)','\g<1>\n<s n="r01"/>\g<2>',l)
        l,hits = re.subn('(\)\.|\.\)|\.</hi>\)|\.</foreign>\))\s+([A-Z])','\g<1>\n<s n="r02"/>\g<2>',l)
        l,hits = re.subn('(</date>\.)\s+([A-Z])','\g<1>\n<s n="r03"/>\g<2>',l)
        l,hits = re.subn('(\]\.|\.\)])\s+([A-Z])','\g<1>\n<s n="r04"/>\g<2>',l)
        l,hits = re.subn('([a-z][a-z]+[\.!?"])\s+([A-Z][a-z]*,\s+[A-Za-z]+[,\s])','\g<1>\n<s n="r05"/>\g<2>',l)
        l,hits = re.subn('([a-z][a-z]+[\.!?"])\s+([A-Z][a-z]*\s+\()','\g<1>\n<s n="r06"/>\g<2>',l)
        l,hits = re.subn('(\.</hi>\])\s+([A-Z])','\g<1>\n<s n="r07"/>\g<2>',l)
        l,hits = re.subn('(\.)\s+(In\s+<date)','\g<1>\n<s n="r08"/>\g<2>',l)
        l,hits = re.subn('(\.</foreign>)\s+([A-Z])','\g<1>\n<s n="r09"/>\g<2>',l)
        l,hits = re.subn('(</ref>\]|</bibl>\))\s+([A-Z])','\g<1>\n<s n="r11"/>\g<2>',l)
        l,hits = re.subn('(\.</hi>|</date>\.)\s+([A-Z][a-z]+\s)','\g<1>\n<s n="r12"/>\g<2>',l)
        l,hits = re.subn('(\.")\s+([A-Z])','\g<1>\n<s n="r13"/>\g<2>',l)

    
    matchs = '([a-zA-Z0-9]+)\s+(century)\s+([AB]\.\s*[DC]\.|after\s+Christ|before Christ)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        if(m[3] == 'after Christ'):
            era = 'A.D.'
        elif(m[3] == 'before Christ'):
            era = 'B.C.'
        else:
            era = m[3]
        curkey = m[1] + ' ' + era
        curkey = re.sub('\.\s+','.',curkey)
        if(not curkey in centval):
            print('missingcentval',curkey,l)
            curval = 'noval'
        else:
            curval = centval[curkey]
        l = re.sub(matchs,'<date when-custom="'+curval+'">\g<1> \g<2> <z/>\g<3></date>',l,1)
    l = re.sub('<z/>','',l)
    
    olymppat = '([0-9]+)([a-z]+\s+Olympiad)'
    while(re.search(olymppat,l)):
        m = re.search(olymppat,l)
        ol = int(m[1])
        startyear = -776 + (4*(ol-1))
        endyear = startyear + 3
        yearrange = str(startyear) + '-' + str(endyear)
        l = re.sub(olymppat,'<date when-custom="'+yearrange+'">\g<1><z/>\g<2></date>',l,1)
    l = re.sub('<z/>','',l)
    
    olymppat2 = '(\(Ol\.\s+|Ol\.\s+)([0-9]+)'
    while(re.search(olymppat2,l)):
        m = re.search(olymppat2,l)
        ol = int(m[2])
        startyear = -776 + (4*(ol-1))
        endyear = startyear + 3
        yearrange = str(startyear) + '-' + str(endyear)
        l = re.sub(olymppat2,'(<date when-custom="'+yearrange+'">\g<1><z/>\g<2></date>',l,1)
    l = re.sub('<z/>','',l)
 
    i = 0
    workl = l
    while(re.search('<date when-custom="([\-0-9]+)"',workl)):
        m = re.search('<date when-custom="([\-0-9]+)"',workl)
        i = i + 1
        if(i> 100):
            print('outofcontrol12',l)
            break
        entrydates[curentryid].append(m[1])
        workl = re.sub('<date when-custom="([\-0-9]+)"','',workl,1)

    l,hits = re.subn('([0-9]+)(\s+to\s+<date when-custom="-)','<date when-custom="-\g<1>">\g<1></date>\g<2>',l)
    newdates3 = newdates3 + hits

    if( re.search('<p>\[<hi rend="smallcaps">[^<]+</hi>\.\]</p>',l)):
            print(l,end='',file=outf)
            continue
            

    matchs = '(<hi rend="smallcaps">)([^<]+)(</hi>)(,\s+No\.\s+)([0-9]+)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        curname = m[2]
        curnum = m[5]
        xrefkey = re.sub('[,\s]+','-',curname.lower()) + '-bio-' + curnum
        xrefs = xrefs + 1
        if(xrefkey in entrylens):
            goodxrefs = goodxrefs + 1
            res = 'hit'
            l = re.sub(matchs,'<ref target="'+xrefkey+'">\g<2></z>\g<4>\g<5></ref>',l,1)
        else:
            res = 'fail'
            print('xref',res,curentryid,curname,curnum,xrefkey,xrefs,goodxrefs)
            l = re.sub(matchs,'<hix rend="smallcaps">\g<2>\g<3>\g<4>\g<5>',l,1)
    l = re.sub('</z>','',l)       
    matchs = '(<hi rend="smallcaps">)([^<]+)(</hi>)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        curname = m[2]
        xrefkey1 = re.sub('[,\.\s]+','-',curname.lower()) + '-bio-1'
        xrefkey2 = re.sub('[,\.\s]+','-',curname.lower()) + '-bio-2'
        xrefkey1 = re.sub('[\-]+','-',xrefkey1)
        xrefkey2 = re.sub('[\-]+','-',xrefkey2)
        xrefs = xrefs + 1
        if(xrefkey1 in entrylens):
            if(xrefkey2 in entrylens):
                res ='amb'
                ambxrefs = ambxrefs + 1

            else:
                res = 'hit'
                goodxrefs = goodxrefs + 1
                
            l = re.sub(matchs,'<ref target="'+xrefkey1+'">\g<2></ref>',l,1)
        else:
            res = 'fail'
            l = re.sub(matchs,'<hix rend="smallcaps">\g<2></hi>',l,1)
        if(0 and not res == 'hit'):
            print('xref2',res,curentryid,curname,xrefkey1,xrefs,goodxrefs,ambxrefs)


    l = re.sub('<hix ','<hi ',l)
    
    l = re.sub('(<bibl n="[^"]+ p)\.','\g<1>age',l)
    l = re.sub('(<bibl n="[^"]+ p)p\.','\g<1>ages',l)
    matchs = '\\b(p\.|pp\.)\s+([0-9][0-9\-]*[a-f]*)'
    l = re.sub(matchs,'<ref type="page" n="\g<2>">\g<1> \g<2></ref>',l)
    l = re.sub('(</ref>)([,\s]+seq\.)','\g<2>\g<1>',l)

    l = re.sub('(Orelli|Orell\.)([,\s]+|[,\s]+[Nn]o[s]*\.\s+)([0-9]+)','<bibl n="Orelli Inscr \g<3>">\g<1>\g<2>\g<3></bibl>',l)
    
    matchs = '(Orell[\.i][,\s+]+)<title>([^<]+)</title>(\s+|[,\s]+[Nn]o[s]*\.\s+)([0-9][0-9\-]*)'
    l = re.sub(matchs,'<bibl n="Orelli Inscr \g<4>">\g<1>\g<2>\g<3>\g<4></bibl>',l)
    
    matchs = '(<bibl n="Orelli Inscr\s+)([^>]+>[^<]+</bibl>)(,\s+)([0-9][0-9\-]*)'
    while(re.search(matchs,l)):
        l = re.sub(matchs,'\g<1>\g<2>\g<3>\g<1>\g<4>">\g<4></bibl>',l,1)
        

    matchs = '(vol\.\s+)([ivx]+)'
    
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        volnum = rnums[m[2]]
        l = re.sub(matchs,'<ref type="volume" n="'+str(volnum)+'">\g<1></z>\g<2></ref>',l)
    l = re.sub('</z>','',l)
    
    matchs = '(<ref type="page"[^>]+>[^<]+</ref>[,\s]+)([0-9][0-9\-]*[a-f]*)'
    while(re.search(matchs,l)):
        l = re.sub(matchs,'\g<1><ref type="page" n="\g<2>">\g<2></ref>',l)
        l = re.sub('(</ref>)([,\s]+seq\.)','\g<2>\g<1>',l)
        
    l = re.sub('(</ref>[,\s]+No[s]\.)\s+([0-9]+)','\g<1><ref type="num" n="\g<2>">\g<1> \g<2>',l)

    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+|[0-9]+\s+[0-9]+/[0-9]+)\s+([A-Z][a-z]+|geog[\.a-z]+|heroic|heav[ily]+\-armed|iambic-trimeter|iambic|mercenary|new|of\s+the[a-z]*|sq\.|square|select)\s+([a-z\-]+)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        number = fract2dec(m[1])
        word1 = m[2]
        word2 = m[3]
        if(word2 in countwords):
            l = re.sub(matchs,'<measure quantity="'+number+'" n="\g<2> \g<3>">\g<1> \g<2> \g<3></measure>',l,1)
        l = re.sub(matchs,'\g<1></z> \g<2> \g<3>',l)
    
    l = re.sub('</z>','',l)
    
    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+|[0-9]+\s+[0-9]+/[0-9]+)\s+(M\.\s*P\.)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        number = fract2dec(m[1])
        word1 = re.sub(' ','',m[2])
    
    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+)\s+(vols\.|vol\.)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        number = re.sub(',','',m[1])
        word1 = re.sub(' ','',m[2])

        l = re.sub(matchs,'<measure quantity="'+number+'" n="volumes">\g<1></z> \g<2></measure>',l,1)
    
    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+|[0-9]+\s+[0-9]+/[0-9]+)\s+([a-z\-]+|[A-Z][a-z]+s\\b)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        number = fract2dec(m[1])
        word1 = m[2]

        if(word1 in countwords or re.search('[A-Z].+[a-tv-z]s$',word1)):
            l = re.sub(matchs,'<measure quantity="'+number+'" n="\g<2>">\g<1> \g<2></measure>',l,1)
        l = re.sub(matchs,'\g<1></z> \g<2>',l,1)
    l = re.sub('</z>','',l)
    
    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+)\s*l\.\s*([0-9]+)\s*s\.\s+([0-9]+)\s*d\.'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        pounds = re.sub(',','',m[1])
        l = re.sub(matchs,'<measure n="pounds-shillings-pence">\g<1>l.</z> \g<2>s. \g<3>d.</measure>',l)
    l = re.sub('</z>','',l)

    matchs = '\\b([Ii]n\s+)([0-9]+\.[0-9]+\.[0-9]+|[0-9]+\.[0-9]+)'

    l = re.sub(matchs,'\g<1><ref>\g<2></ref>',l)
    
    l = re.sub('(4to\.|4to)','<measure n="quarto">\g<1></measure>',l)
    l = re.sub('(8vo\.|8vo)','<measure n="octavo">\g<1></measure>',l)
    
    
    matchs = '([0-9]+)°\s+([0-9]+)′\s+([0-9]+)″'
    while(re.search(matchs,l)):
          m = re.search(matchs,l)
          print(curentryid,m[1],m[2],m[3])
          l = re.sub(matchs,'<measure type="degree" quantity="\g<1>">\g<1></z>°</measure> <measure type="minute" quantity="\g<2>">\g<2></z>′</measure> <measure type="second" quantity="\g<3>">\g<3></z>″</measure>',l)
          
    l = re.sub('</z>','',l)      
          
    matchs = '([0-9]+)(\s+or\s+|\s+and\s+|\s+to\s+)<measure quantity="([^"]+)" n="([^"]+)">'

    l = re.sub(matchs,'<measure quantity="\g<1>" n="\g<4>">\g<1></measure>\g<2><measure quantity="\g<3>" n="\g<4>">',l)

    
    m = re.search('<bibl n="([A-Z][a-z]+)',l)
    if(m):
        if(not m[1] in lastbib):
            lastbib.append( m[1])
        
    m = re.search('<date when-custom="([^"]+)"',l)
    if(m):
            lastdate = m[1]
            

    
    matchs = '\\b([Ii]n\s+)([0-9]+\.[0-9]+\.[0-9]+|[0-9]+\.[0-9]+)'

    l = re.sub(matchs,'\g<1><ref>\g<2></ref>',l)
    

    
    m = re.search('entry" xml:id="([^"]+)',l)
    if(m):
        curentryid = m[1]
        lastdate = 'nolast'
        lastbib = []
    
    matchs = '\\b([a-zA-Z]+)\s+([a-zA-Z]+)\s+\(([0-9]+)\)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
    
        if(m and int(m[3]) > 9 and int(m[3]) < 1000 and not m[2] in notyears):
            j = j + 1
            if(re.search('^-',lastdate) or 'Liv' in lastbib or 'Cic' in lastbib or 'Thuc' in lastbib or 'Dionys' in lastbib):
                newdate = '-' + m[3]
            else:
                newdate = m[3]
            print(j,'paren',curentryid,lastdate,'Cic' in lastbib,lastbib,m[1],m[2],m[3],newdate)
            l = re.sub(matchs,'\g<1> \g<2> (<date when-custom="'+newdate+'">\g<3></date>)',l,1)
        else:
            l = re.sub(matchs,'\g<1> \g<2> (<ref>\g<3></ref>)',l,1)
    
    matchs = '\\b([Ii]n)\s([0-9]+)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)            
        newdate = m[2]
        if(re.search('^-',lastdate) or  'Plb' in lastbib or 'Liv' in lastbib or 'Cic' in lastbib):
            if(int(m[2])< 900):
                newdate = '-' + m[2]
            else:
                newdate = m[2]
        else:
            newdate = m[2]
        if(len(newdate) == 1):
            shows = l
        else:
            shows = ''
        i = i + 1
        if(lastdate == 'nolast' and not lastbib == 'nobib' and int(newdate)  < 1000):
            print(i,curentryid,'newdate',lastdate,lastbib,newdate,shows)
        l =re.sub(matchs,'\g<1> <date when-custom="'+newdate+'">\g<2></date>',l,1)
    
    
    
    print(l,end='',file=outf)
outf.close()

print('done')


badn p-consentius-bio-1.2-1 p-consentius-bio-1.2 1        <p>2. <hi rend="smallcaps">CONSENTIUS</hi>, who rose to high honour under Valentinian III., by whom he was named Comes Palatii and despatched upon an important mission to Theodosius. He also had a son, namely</p>

badn p-consentius-bio-1.3-1 p-consentius-bio-1.3 1        <p>3. <hi rend="smallcaps">CONSENTIUS</hi>, who devoted himself to literary leisure and the enjoyments of a rural life, and is celebrated as well as his grandfather by Sidonius.</p>

unmatched 374 6 376
unmatched 377 8 378
unmatched 653 4 654
unmatched 728 9 729
unmatched 374 6 376
1 paren agesilaus-ii-bio-1 -398 False [] third year 396 -396
2 paren ahenobarbus-bio-7 -58 False ['Plin'] into Italy 49 -49
3 paren ahenobarbus-bio-7 -58 False ['Plin', 'Suet'] of Pharsalia 48 -48
4 paren ahenobarbus-bio-8 -49 False [] of Pharsalia 48 -48
5 paren ahenobarbus-bio-8 -49 False [] of Caesar 44 -44
6 paren ahenobarbus-bio-8 -49 False [] of Philippi 42 -42
7 paren alaricus-

xref fail granius-bio-1 CRASSUS 23 crassus-bio-23 3435 973
xref fail gregorius-jcm-bio-4 GEORGIUS PERIPATETICUS 41 georgius-peripateticus-bio-41 3463 985
xref fail gregorius-jcm-bio-31 GEORGIUS 41 georgius-bio-41 3494 992
83 paren hamilcar-bio-7 -262 False ['Zonar'] following year 256 -256
84 paren hannibal-bio-7 -250 False ['Plb'] following year 249 -249
85 paren hannibal-bio-10 -247 False ['Plb'] next year 220 -220
86 paren hannibal-bio-10 -213 False ['Plb', 'Liv', 'Gel', 'Zonar'] next year 212 -212
87 paren hannibal-bio-10 -213 False ['Plb', 'Liv', 'Gel', 'Zonar'] ensuing spring 211 -211
88 paren hannibal-bio-10 -213 False ['Plb', 'Liv', 'Gel', 'Zonar'] next season 210 -210
89 paren hannibal-bio-10 -213 False ['Plb', 'Liv', 'Gel', 'Zonar'] following summer 209 -209
90 paren hannibal-bio-10 -213 False ['Plb', 'Liv', 'Gel', 'Zonar'] following summer 208 -208
91 paren hannibal-bio-10 -213 False ['Plb', 'Liv', 'Gel', 'Zonar'] ensuing year 207 -207
92 paren hannibal-bio-10 -193 False ['P

148 paren michael-ii-balbus-bio-1 820 False [] at Antioch 821 821
149 paren michael-iii-bio-1 842 False [] the Paulicians 848 848
150 paren michael-iii-bio-1 842 False [] private life 854 854
151 paren michael-iii-bio-1 842 False [] the purpose 866 866
152 paren michael-iii-bio-1 842 False [] same year 866 866
1 michael-bio-9 newdate nolast [] 877 
153 paren mithridates-vi-bio-1 -68 False ['Just', 'App', 'Plut'] following year 66 -66
154 paren mithridates-vi-bio-1 -68 False ['Just', 'App', 'Plut'] following year 65 -65
xref fail murcus-l-statius-bio-1 LONGINUS 11 longinus-bio-11 5984 1762
155 paren narses-bio-3 472 False [] recalled Narses 539 539
156 paren narses-bio-3 472 False [] of grief 568 568
1 nepotianus-flavius-popilius-bio-1 newdate nolast ['Eutrop'] 301 
xref fail nestorius-bio-1 DOROTHEUS 5 dorotheus-bio-5 6112 1806
xref fail nestorius-bio-1 Meletius 7 meletius-bio-7 6124 1809
157 paren nicephorus-i-bio-1 802 False [] into Bulgaria 811 811
1 nicephorus-bio-3 newdate nolast 

In [2]:
print('newdates1',newdates1,'newdates',newdates,'newdates2',newdates2,'sents',sentences)

i = 0
for foo in sorted(entrylens,key=entrylens.get,reverse=True):
    #if(1 and len(entrydates[foo])>0):
   #     continue
    i = i + 1
    if(i>1000):
        continue
    #if(entrylens[foo] < 1000):
    #    continue
    if(len(entrydates[foo]) == 0):
        print(i,entrylens[foo],len(entrydates[foo]),foo,entryheads[foo],entrydates[foo])
        
        
        
        
        

newdates1 922 newdates 510 newdates2 59 sents 0
17 9795 0 heracles-bio-1 Heracles or HERCULES  []
103 3853 0 romulus-bio-1 Ro'mulus  []
131 3357 0 dionysus-bio-1 Diony'sus  []
143 3172 0 odysseus-bio-1 Odysseus  []
165 2947 0 cabeiri-bio-1 CABEIRI  []
204 2542 0 argonautae-bio-1 Argonautae  []
224 2376 0 appuleius-bio-10 Appuleius  []
229 2347 0 epictetus-bio-1 Epicte'tus  []
232 2341 0 avienus-rufus-festus-bio-1 Avie'nus, Rufus Festus  []
236 2314 0 hermes-bio-1 Hermes  []
240 2270 0 zeus-bio-1 Zeus  []
253 2223 0 artemis-bio-1 A'rtemis  []
304 1966 0 poseidon-bio-1 Poseidon  []
308 1941 0 aeneas-bio-1 Aene'as  []
317 1880 0 nepos-cornelius-bio-1 Corne'lius Nepos  []
322 1859 0 dictys-cretensis-bio-1 Dictys Cretensis  []
329 1838 0 agamemnon-bio-1 Agamemnon  []
332 1831 0 jason-bio-1 Jason  []
335 1821 0 aenesidemus-bio-2 Aeneside'mus  []
343 1791 0 musae-bio-1 Musae  []
344 1781 0 io-bio-1 Io  []
346 1757 0 achilles-bio-1 Achilles  []
365 1684 0 diomedes-bio-1 Diome'des  []
366 1684 

In [4]:
print(len(entrylens))

19902


In [10]:
abbrevs = {}
f = open('smithbio-work.xml')

for l in f:
    if(not re.search('<p>',l)):
        continue
    l = re.sub('<bibl[^>]+>[^<]+</bibl>',' ',l)
    l = re.sub('<title[^>]+>[^<]+</title>',' ',l)
    while(re.search('([A-Z][a-z][a-z][a-z]\.)',l)):
        m = re.search('([A-Z][a-z][a-z][a-z]\.)',l)
        curabb = m[1]
        if(curabb == 'Hist.'):
            print(l)
        if(curabb in abbrevs):
            abbrevs[curabb] = abbrevs[curabb] + 1
        else:
            abbrevs[curabb] = 1
        l = re.sub('([A-Z][a-z][a-z][a-z]\.)','',l)
    

f.close()

for foo in sorted(abbrevs,key=abbrevs.get,reverse=True):
    print(abbrevs[foo],foo)

<p>2. The illustrious (<foreign xml:lang="grc">Ἰλλούστριος</foreign>), the author of an epigram in the Greek Anthology ( ) " on the quoit of Asclepiades." Nothing more is known of him, unless he be the same person as Ablabius, the Novatian bishop of Nicaea, who was a disciple of the rhetorician Troilus, and himself eminent in the same profession, and who lived under Honorius and Theodosius II., at the end of the fourth and the beginning of the fifth centuries after Christ. (Socrates, <title>Hist. Eccl.</title> 7.12.) </p> 

<p>3. The One-eyed (<foreign xml:lang="grc">ὁ Μονόφθαλμος</foreign>), the pupil and successor in the See of Caesarea of Eusebius <date when-custom="340">A. D. 340</date>, whose life he wrote. (Socrates, <title>Hist. Eccl.</title> 2.4.)

<p>Freind's <title>Hist. of Physic</title>; Sprengel, <title>Hist. de la Méd.</title>; Haller, <title>Biblioth. Medic. Pract.</title>; Barchusen, <title>Hist. Medic.</title></p> 

<p>(<persName xml:lang="grc"><surname full="yes">Ἀδαμ

<p>Nitzsch, <title>Hist. Homeri</title>, i. <ref type="page" n="88">p. 88</ref>; Bernhardy, in his edition of <title>Dionys. Perieg.</title> <ref type="page" n="498">p. 498</ref>, &amp;c., and <hi rend="ital">ad Suidam</hi>, i. <ref type="page" n="1395">p. 1395</ref>; <hi rend="ital">Lobeck, Aglaoph.</hi> ii. <ref type="page" n="990">p. 990</ref>,&amp;c.; Welcker, <hi rend="ital">Der Epische Cyclus</hi>, <ref type="page" n="75">p. 75</ref>, &amp;c.</p> 

<p>For further information respecting Dioscorides and the editions of his work, see Le Clerc, <title>Hist. de la Méd.</title>; Haller, <title>Biblioth. Bolan.</title>; Sprengel, <title>Hist. de la Méd.</title>; Fabric. <title>Biblioth. Graeca</title>; Bostock's <hi rend="ital">History of Medicine</hi>; Choulant, <hi rend="ital">Handbuch der Bücherkunde für die Aeltere Medicin.</hi></p> 

<p> (<abbr>Ath.</abbr> vi. <ref type="page" n="254e">p. 254e</ref>.; and perhaps in  , <foreign xml:lang="grc">Διφίλου</foreign> should be substituted

<p>Cave, <title>Hist. Lit.</title> ad an. 869; Fabric. <title>Bibl. Graec.</title> vol. xi. <ref type="page" n="189">p. 189</ref>.</p> 

<p>Cave, <title>Hist. Lit.</title> ad an. 1440; Fabric. <title>Bibl. Graec.</title> vol. xi. <ref type="page" n="189">p. 189</ref>.</p> 

<p>Cave, <title>Hist. Lit.</title> ad an. 1043; Fabric. <title>Bibl. Graec.</title> vol. xi. <ref type="page" n="195">pp. 195</ref>, <ref type="page" n="196">196</ref>.</p> 

<p>Cave, <title>Hist. Lit.</title> ad an. 878; Fabric. <title>Bibl. Graec.</title> vol. xi. <ref type="page" n="205">p. 205</ref>.</p> 

<p>Vossius, <hi rend="ital">de Hist. Graec.</hi> <ref type="page" n="178">p. 178</ref>, ed. Westermann; Clinton, <hi rend="ital">F. H.</hi> vol. iii. <ref type="page" n="534">p. 534</ref>; Jahn, <hi rend="ital">de Palamede</hi>, <ref type="page" n="31">p. 31</ref>; and more especially Preller, in the <title>Zeitschrift für die Alterthumswissenschaft,</title> 1846, <ref type="page" n="673-688">pp. 673-688</ref>

<p>(<persName xml:lang="grc"><surname full="yes">Θεόδοκος</surname></persName>), the name given by Pococke (in his Latin Version of Abú-l-Faraj, <title>Hist. Dynast.</title> <ref type="page" n="128">p. 128</ref>), and Wüstenfeld (<title>Gesch. der Arab. Aerzte</title>, <ref type="page" n="9">p. 9</ref>) to a Greek physician in the service of Hajáj Ibn Yúsuf, the general of the chalif 'Abdu-l-Malek Ibn Merwán, in the <date when-custom="601-700">seventh century after Christ</date>.

<p>Garnier, <hi rend="ital">Dissertationes</hi>, in vol. 5 of Schulze's edition; Tillemont, <hi rend="ital">Mém.</hi> vol. xiv.; Cave, <title>Hist. Litt. s. a. 423</title>, <ref type="page" n="405">pp. 405</ref>, foll., ed. Basil.; Fabric. <title>Bibl. Graec.</title> vol. vii. <ref type="page" n="429">pp. 429</ref>, foil., vol. viii. <ref type="page" n="277">pp. 277</ref>, foil.; Schulze, <hi rend="ital">De Vita et Scriptis B. Theodoreti Dissertatio</hi>, prefixed to vol. i. of his edition ; Neander, <hi rend

In [15]:
titles = {}
f = open('smithbio-work.xml')

for l in f:
    if(not re.search('<p>',l)):
        continue
    l = re.sub('<bibl[^>]+>[^<]+</bibl>',' ',l)
    l = re.sub('<title[^>]+>[^<]+</title>',' ',l)
    while(re.search('<hi rend="ital">([A-Z][a-z]+\. [^<]+)',l)):
        m = re.search('<hi rend="ital">([A-Z][a-z]+\. [^<]+)',l)
        print(l)
        curabb = m[1]
        if(curabb in titles):
            titles[curabb] = titles[curabb] + 1
        else:
            titles[curabb] = 1
        l = re.sub('<hi rend="ital">([A-Z][a-z]+\. [^<]+)','',l)
    

f.close()

for foo in sorted(titles,key=titles.get,reverse=True):
    print(titles[foo],foo)

In [1]:
import re

xrefs = {}
f = open('smithbio-work.xml')

xrefcnt = 0
for l in f:
    if(not re.search('<p>',l)):
        continue
    l = re.sub('<bibl[^>]+>[^<]+</bibl>',' ',l)
    l = re.sub('<title[^>]+>[^<]+</title>',' ',l)
    while(re.search('\[<hi rend="smallcaps">([^<]+)',l)):
        m = re.search('\[<hi rend="smallcaps">([^<]+)',l)
        #print(l)
        curabb = m[1]
        if(curabb in xrefs):
            xrefs[curabb] = xrefs[curabb] + 1
        else:
            xrefs[curabb] = 1
        l = re.sub('\[<hi rend="smallcaps">([^<]+)','',l)
    

f.close()

for foo in sorted(xrefs,key=xrefs.get,reverse=True):
    print(xrefs[foo],foo)

10 MAXIMUS.
9 GALLUS.
8 FLACCUS.
8 RUFUS
5 MACER.
5 CONSTANTINUS VII.
5 GEORGIUS
5 LEPIDA
5 MAXIMUS
5 FLACCUS
5 PRISCUS
4 FUSCUS.
4 CLUENTIUS.
4 LABEO.
4 CAPITOLINUS
4 COBIDAS.
4 MAMMAS.
4 STUDITA.
4 LONGINUS
4 SEVERUS
4 SABINUS
3 MILO.
3 NATALIS.
3 CRASSUS.
3 LARGUS
3 VERRES.
3 CHRYSOLORAS.
3 CATO
3 VARRO
3 PARDUS.
3 CALECAS.
3 DAMASCENUS
3 CIVILIS
3 DIOCLETIANUS
3 EUTHYMIUS
3 TURPILIANUS.
3 GALLUS
3 BASSUS
3 SERENUS
3 SECUNDUS
3 BAREA
3 SILANUS
3 GALLICANUS
3 QUADRATUS
3 PROCULUS
3 POLLIO
3 SATURNINUS
2 ARBORIUS.
2 DEXTER.
2 JUVENTINUS.
2 ANTISTATES.
2 POLEMO.
2 MUTILUS.
2 REGULUS.
2 BATTUS
2 FORTUNATIANUS.
2 ACCIUS
2 TUCCA.
2 AVIENUS.
2 NEMESIANUS.
2 CORDUS.
2 CRISPINILLA.
2 VARENUS.
2 VESPASIANUS.
2 CIVILIS.
2 TACITUS.
2 CODINUS.
2 SAXA
2 PEPAGOMENUS.
2 BALBUS
2 FULGENTIUS.
2 FRONTINUS.
2 GEMISTUS.
2 PLAUTIANUS
2 PALAMAS.
2 GETA.
2 ANAGNOSTES.
2 HESYCHIUS
2 CAMATERUS.
2 CAMENIATA.
2 CLIMACUS.
2 EUGENICUS
2 PHILOPONUS.
2 BRYENNIUS.
2 FIRMICUS.
2 VESTINUS.
2 VALENTINIANUS.
2 LEO VI.


In [2]:
import re

centuries = {}
    
def cntcents(fname):
    f = open(fname)

    xrefcnt = 0
    for l in f:
        if(not re.search('<p>',l)):
            continue
        l = re.sub('<bibl[^>]+>[<]+</bibl>',' ',l)
        l = re.sub('<title[^>]+>[<]+</title>',' ',l)
        matchs = '([a-zA-Z0-9]+)\s+(century)\s+([AB]\.\s*[DC]\.)'
        while(re.search(matchs,l)):
            m = re.search(matchs,l)
        #print(l)
            curabb = m[1] + ' ' + m[3]
            curabb = re.sub('\.\s+','.',curabb)
            if(curabb in centuries):
                centuries[curabb] = centuries[curabb] + 1
            else:
                centuries[curabb] = 1
            l = re.sub(matchs,'',l)
    

    f.close()
cntcents('smithbio-work.xml')
cntcents('../002/smithgeo-work.xml')

for foo in sorted(centuries):
    if(re.search('B',foo)):
        print('centval["'+foo+'"] = "-100-101"')
    else:
        print('centval["'+foo+'"] = "101-100"')

centval["12th A.D."] = "101-100"
centval["4th A.D."] = "101-100"
centval["4th B.C."] = "-100-101"
centval["6th B.C."] = "-100-101"
centval["7th A.D."] = "101-100"
centval["7th B.C."] = "-100-101"
centval["9th A.D."] = "101-100"
centval["eighth B.C."] = "-100-101"
centval["eleventh B.C."] = "-100-101"
centval["fifth A.D."] = "101-100"
centval["fifth B.C."] = "-100-101"
centval["first A.D."] = "101-100"
centval["first B.C."] = "-100-101"
centval["fourth B.C."] = "-100-101"
centval["last B.C."] = "-100-101"
centval["ninth B.C."] = "-100-101"
centval["second B.C."] = "-100-101"
centval["seventh B.C."] = "-100-101"
centval["sixth A.D."] = "101-100"
centval["sixth B.C."] = "-100-101"
centval["tenth B.C."] = "-100-101"
centval["third A.D."] = "101-100"
centval["third B.C."] = "-100-101"
centval["twelfth B.C."] = "-100-101"


In [20]:

def getbestentry(query):
    global singnameindex1
    query = re.sub('^([A-Za-z])\.\s+(.+)','\g<2> \g<1>',query)
    query = re.sub('[\,\.\s]+','-',query)
    queries = query.lower().split('-')
    if(len(queries)> 1):
        filts = '\\b' + queries[1] + '\\b'
        #print('filts',filts)
    else:
        filts = ''
    if(not queries[0] in singnameindex1):
        #print(queries[0],'not known')
        return
    workdict = singnameindex1[queries[0]]
    print('q0',queries[0])
    retlist = sorted(workdict,key=workdict.get,reverse=True)
    
    #print(retlist)
    goods = []
    bads = []
    for foo in sorted(workdict,key=workdict.get,reverse=True):
        texts = entryheads[foo].lower()
        texts = re.sub("'",'',texts)
        #print(filts,'searching',foo,texts)
        if(filts and not re.search(filts,texts)):
            #continue
            #print('\tfailfilt',end=' ')
            bads.append(foo)
        else:
            goods.append(foo)
        #print(foo,workdict[foo])
    print('good',goods)
    for foo in goods:
        print(foo,entrylens[foo],entryheads[foo],firstparatexts[foo])
    print('failed',bads)

firstparatexts = {}

f = open('smithbio-work.xml')
curentryid = ''
firstpara = 0
for l in f:
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        curentryid = m[1]
        firstpara = 1
    if(re.search('<p>',l)):
        if(firstpara):
            workl = l.lower()
            workl = re.sub('<bibl n="([^"]+)">[^<]+</bibl>','\g<1>',workl)
            workl = re.sub('<[^>]+>',' ',workl)
            firstparatexts[curentryid] = workl
        firstpara = 0


singnameindex1 = {}
for foo in entrylens:
    workid = re.sub('-bio.*','',foo)
    if( not foo in entryheads):
        print('nohead',foo)
        continue
    headtext = re.sub('<[^>]+>',' ',entryheads[foo].lower())
    headtext = re.sub('[\.;:!,\-]+',' ',headtext)
    headtext = re.sub("([a-z])'([a-z])",'\g<1>\g<2>',headtext)
    #print(headtext)
    for tmpkey in headtext.split():
    #for tmpkey in workid.split('-'):
        if(len(tmpkey)<3):
            continue
        if(not tmpkey in singnameindex1):
            singnameindex1[tmpkey] = {}
        singnameindex1[tmpkey][foo] = entrylens[foo]


getbestentry('philippus ii')

nohead tiridates-bio-1
q0 philippus
good ['philippus-ii-bio-1', 'philippus-ii-m-julius-bio-1']
philippus-ii-bio-1 6432 Philippus Ii.   (  φίλιππος  ), the 18th king of  macedonia , if we count from caranus, was the youngest son of amyntas ii. and eurydice, and was born in  b. c. 382 .

philippus-ii-m-julius-bio-1 234 Philippus Ii., M. Ju'lius   son of the foregoing, was a boy of seven at the accession ( a. d. 244 ) of his father, by whom he was forthwith proclaimed caesar, and three years afterwards ( 247 ) chosen consul, being at the same time admitted to share the purple with the title of augustus.

failed ['philippus-v-bio-1', 'philippus-marcius-bio-5', 'philippus-liteccles-bio-26', 'philippus-i-m-julius-bio-1', 'philippus-liteccles-bio-27', 'philippus-marcius-bio-3', 'philippus-marcius-bio-6', 'philippus-physician-bio-4', 'philippus-bio-30', 'philippus-liteccles-bio-19', 'philippus-liteccles-bio-22', 'philippus-liteccles-bio-16', 'philippus-liteccles-bio-8', 'philippus-liteccles-bi

In [5]:
import re
f = open('smithbio-work.xml')

i = 0
j = 0
k = 0
firstpara = 0
for l in f:
    m = re.search('entry"\s+xml:id="([^"]+-)([0-9abc]+|group|gens|geo)"',l)
    if(m):
        curentryid = m[1]+m[2]
        firstpara = 1
        expectnum = m[2]
        basename = re.sub('(-bio|-group).*$','',curentryid)
    
    
    #if(re.search('<head>',l)):
   #     workl = re.sub('</persName>[^<]+<persName[^>]+>',' ',l)
   # else:
    #    workl = l
    m = re.search('(<head>|<head><label[^>]*>)<persName[^>]*>(.+)</persName>',l)
    if(m):
        l = re.sub('<note[^>]*>.+</note>',' ',l)
        curname = l.lower()
        curname = re.sub('</persname>[^<]+<persName[^>]*>',' ',curname)
        curname = re.sub('<[^>]+>','',curname)
        curname = re.sub("'",'',curname)
        curname = re.sub('[\s\.,]+','-',curname)
        curname = re.sub('-$','',curname)
        if(curentryid == 'cleon-bio-1'):
            print(curname,l,sep='---')
        nameargs = curname.split('-')
        idargs = basename.split('-')
        extraargs = []
        for foo in idargs:
            if(not foo in nameargs):
                extraargs.append(foo)
        #if(not curname == basename):
        if(extraargs):
            k = k + 1
            print('mismatch',k,curentryid,curname,extraargs)
    m = re.search('([A-Z][a-z]+\s+[Gg]ens)',l)
    if(firstpara and re.search('-1$',curentryid) and m):
        curgens = m[1]
        #print(j,'gens',curentryid,curgens,l)
        
        
    m = re.search('<p>([0-9]+)\.',l)
    if(1 and m and firstpara and not m[1] == expectnum):
        i = i + 1
        #print(i,curentryid,l)
        
    if(re.search('<p>',l)):
        firstpara = 0

f.close()

mismatch 1 acherusia-geo-1 acherusia ['geo', '1']
mismatch 2 appuleia-gens-bio-1 appuleia ['gens']
mismatch 3 artavasdes-bio-3 artavasde-or-artabasdus ['artavasdes']
mismatch 4 asclepiades-areius-bio-3 asclepiades ['areius']
mismatch 5 callias-fam-2 callias-and-hipponicus ['fam', '2']
mismatch 6 callias-bio-3 hipponicus-i ['callias']
mismatch 7 callias-bio-5 hipponicus-ii-or-hipponicus-ammon ['callias']
mismatch 8 callias-bio-7 hipponicus-iii ['callias']
mismatch 9 camerinus-gens-bio-1 camerinus ['gens']
mismatch 10 cato-porcius-gens-bio-1 cato-porcius ['gens']
mismatch 11 claudius-gens-bio-1 claudius ['gens']
mismatch 12 claudius-bio-41 p-clodius-pulcher ['claudius']
cleon---<head><persName xml:lang="la"><surname full="yes">Cleon</surname></persName></head> 

mismatch 13 crassus-licinius-bio-11 licinia ['crassus', 'licinius']
mismatch 14 diodorus-tarsensis-bio-17 diodorus-of-tarsus ['tarsensis']
mismatch 15 gregorius-antiochus-bio-12 gregorius-of-antioch ['antiochus']
mismatch 16 jupi

In [4]:
l = '<head><label><persName xml:lang="la"><surname full="yes">Michael</surname> <genName full="yes">I.</genName></persName> or <persName xml:lang="la"><surname full="yes">Michael</surname> <addName full="yes">Rhanga\'be</addName></persName></label></head>'
l = re.sub('<persName[^>]*>(.+)</persName>.*','\g<1>',l)

print(l)


<head><label><surname full="yes">Michael</surname> <genName full="yes">I.</genName></persName> or <persName xml:lang="la"><surname full="yes">Michael</surname> <addName full="yes">Rhanga'be</addName>


In [28]:
f = open('smithbio-work.xml')

firstpara = 0
for l in f:
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        curentryid = m[1]
        firstpara = 1
    if(re.search('<p>',l) and firstpara):
        firstpara = 0
        if(re.search('(family of|family[\-\s]+name|cognomen|agnomen| surname\\b)',l) and not re.search('(family|gens|cognomen|agnomen|surname)',curentryid)):
            print(curentryid,l)
            

f.close()

abaeus-bio-1 <p>(<persName xml:lang="grc"><surname full="yes">Ἀβαῖος</surname></persName>), a surname of Apollo derived from the town of Abae in Phocis, where the god had a rich temple. (Hesych. <hi rend="ital">s. v.</hi> <foreign xml:lang="grc">Ἄβαι</foreign>; <bibl n="Hdt. 8.33">Hdt. 8.33</bibl>; <bibl n="Paus. 10.35.1">Paus. 10.35.1</bibl>, &amp;c.) </p> 

abrettenus-bio-1 <p>(<label xml:lang="grc">Ἀβρεττηνός</label>), a surname of Zeus in Mysia. (<bibl n="Strabo xii.p.574">Strab. xii. <ref type="page" n="574">p. 574</ref></bibl>.) </p> 

acacesius-bio-1 <p>(<persName xml:lang="grc"><surname full="yes">Ἀκακήσιος</surname></persName>), a surname of Hermes (Callim. <title>Hym. in Dian.</title> 143), for which Homer (<bibl n="Hom. Il. 16.185">Hom. Il. 16.185</bibl>; <bibl n="Hom. Od. 24.10">Od. 24.10</bibl>) uses the form <foreign xml:lang="grc">ἀκάκητα</foreign> (<foreign xml:lang="grc">ἀκακήτης</foreign>).

acersecomes-bio-1 <p>(<label xml:lang="grc">Ἀκερσεκόμης</label>), a surname o

catus-bio-1 <p>a word indicating shrewdness, caution, sagacity, or the like, was a surname of <abbr>Sex.</abbr> Aelius Paetus, who was consul <date when-custom="-198">B. C. 198</date> [<ref target="paetus-bio-1">PAETUS</ref>], and the cognomen of <abbr>Sex.</abbr> Aelius, consul in <date when-custom="4">A. D. 4</date>, with C. Sentius Saturninus. (<bibl n="Vell. 2.103">Vell. 2.103</bibl>.)</p> 

caudinus-bio-1 <p>a surname of several of the Cornelii Lentuli. [<ref target="lentulus-bio-3">LENTULUS.</ref>] <pb n="657" source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044099890501%25seq=675"/></p> 

causius-bio-1 <p>(<persName xml:lang="grc"><surname full="yes">Καούσιος</surname></persName>), a surname of Asclepius, derived from Caus in Arcadia, where he was worshipped. (Steph. <abbr>Byz.</abbr> <hi rend="ital">s. v.</hi> <foreign xml:lang="grc">Καοῦς</foreign>; comp. <bibl n="Paus. 8.25.1">Paus. 8.25.1</bibl>.) </p> 

cenaeus-bio-1 <p>(<persName xml:lang="grc"><surname full="yes">Κηνα

hegesaratus-bio-1 <p>was descended from an ancient and noble family of Larissa in Thessaly, and was leader of the Pompeian party in that city during the civil war in <date when-custom="-48">B. C. 48</date>.

heliodorus-bio-14 <p>The author of the oldest and by far the best of the Greek romances, Heliodorus, the son of Theodosius, was a native of Syria, and was born, not, as Photius says, at Aminda, but at Emesa, as he himself tells us at the <pb n="374" source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044011634888%25seq=400"/> end of his romance :--<foreign xml:lang="grc">Τοιόνδε πέρας ἔσχε τὸ σύνταγμα τῶν περὶ Θεαγένην καὶ Χαρίκλείκλειαν Αἰθιοπικῶν ὁ συνέταξεν ἀνὴρ Φοίνιξ Ἐμεσηνὸς, τῶν ἀφʼ Ἡλίου, γένος, Θεοδοσίον παῖς Ἡλιόδωρος</foreign>. The words <foreign xml:lang="grc">τῶν ἀφʼ Ἡλίου γενος</foreign> no doubt mean that he was of the family of priests of the Syrian god of the Sun (Elagabalus).

hellotia-bio-1 <p>or HELLO'TIS (<foreign xml:lang="grc">Ἐλλωτία</foreign> or <foreign x

nero-bio-1 <p>was a cognomen of the Claudia Gens, which is said to signify, in the Sabine tongue "fortis ac strenuus." (Sueton. <title>Tib. Nero</title>, 1; and the remarks of Gellius, <bibl n="Gel. 13.22">13.22</bibl>.)</p> 

nesiotes-bio-1 <p>a sculptor, appears to have been an assistant of the celebrated Athenian artist Critias, and not a surname of the latter, as some modern writers have conjectured. [<ref target="critias-bio-1">CRITIAS</ref>, <abbr>Vol.</abbr> I. <ref type="page" n="893">p. 893</ref>.]</p> 

nicanor-bio-18 <p>5. Stephanus Byzantinus mentions a writer of this name to whom he adds that of <foreign xml:lang="grc">Λέανδρος</foreign>, as the author of a work called <foreign xml:lang="grc">Μετονοομασιας</foreign>. Athenaeus quotes the same work, but calls the writer a Cyrenian, without giving him the surname.

nice-bio-3 <p>3. Nice also occurs as a surname of Athena, under which the goddess had a sanctuary on the acropolis of Megara. (<bibl n="Paus. 1.42.4">Paus. 1.42.4

scipio-bio-7 <p>6. <persName xml:lang="la"><forename full="yes">Cn.</forename> <surname full="yes">Cornelius</surname> <addName full="yes">Scipio</addName> <addName full="yes">Asina</addName></persName>, the son of No. 5. The reason of his cognomen Asina is related by Macrobius (<bibl n="Macr. 1.6">Macr. 1.6</bibl>).

sciras-bio-2 <p>(<label xml:lang="grc">Σκιράς</label>), a surname of Athena, under which she had a temple in the Attic port of Phaleron, and in the island of Salamis (<bibl n="Paus. 1.1.4">Paus. 1.1.4</bibl>; <bibl n="Hdt. 8.94">Hdt. 8.94</bibl>).

scopas-bio-2 <p>(<persName xml:lang="grc"><surname full="yes">Σκόπας</surname></persName>), one of the most distinguished sculptors of the later Attic school, was a native of Paros, which was then subject to Athens (<bibl n="Strabo xiii.p.604">Strab. xiii. p.604</bibl>; <bibl n="Paus. 8.45.4">Paus. 8.45.4</bibl>); and he appears to have belonged to a family of artists in that island.

seleucus-iii-bio-1 <p>(<persName xml:lang="

In [46]:
import re

trigrams = {}
firstpara = 0
curentryid = ''
lines = 0
f = open('smithbio-work.xml')
for l in f:
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        curentryid = m[1]
        firstpara = 1
        continue
    if(re.search('<p>',l) and firstpara):
        text = re.sub('<[^>]+>',' ',l)
        text = re.sub('^[\s\(]+[^()]+\)[,\s]*','',text)
        searchtext = text[0:100].lower()
        if(curentryid == 'nicephorus-i-bio-1'):
            print(curentryid,'test',searchtext,l)
        if( re.search('(empress|emperor)',searchtext) and re.search('(constantinople|the\s+east)',searchtext)):
            print(curentryid,'emperor of constantinople',text)
        firstpara = 0
        continue
        
        m = re.search('([A-Z][a-z]+\s+[A-Z][a-z]+\s+[A-Z][a-z]+)',l)
        if(m):
            print(lines,curentryid,'trigram',m[1])
        m = re.search('([A-Z][a-z]+\s+[A-Z][a-z]+)',l)
        if(m):
            print(lines,curentryid,'bigram',m[1])
        l = re.sub('<bibl[^>]+>[^<]+</bibl>',' ',l)
        l = re.sub('<date[^>]+>[^<]+</date>',' DATE ',l)
        if(re.search('[a-z][a-z]',text)):
        
            #print(curentryid,firstpara,text)
            text = re.sub('(of|of\s+the)\s+([a-z]+)\s+([A-Z])','\g<1>-\g<2>-\g<3>',text)
            text = re.sub('(of|of\s+the)\s+([A-Z])','\g<1>-\g<2>',text)
            text = re.sub('(of)\s+(the)','\g<1>-\g<2>',text)
            text = re.sub('([A-Z][a-z]+)\s+([A-Z])','\g<1>-\g<2>',text)
            words = text.split()
            prev1 = ''
            prev2 = ''
            for foo in words:
                if(prev2 and prev1):
                    trigram = prev2 + ' ' + prev1 + ' ' + foo
                    if(trigram in trigrams):
                        trigrams[trigram] = trigrams[trigram]  + 1
                    else:
                        trigrams[trigram] = 1
                if(re.search('^[A-Z].+[^,]$',prev2) and re.search('^of',prev1)):
                    print(lines,'ofpat',curentryid,prev2,prev1,foo)
                prev2 = prev1
                prev1 = foo
            firstpara = 0
        else:
            continue
    lines = lines + 1

print('done',lines)

i = 0
for foo in sorted(trigrams,key=trigrams.get,reverse=True):
    i = i + 1
    if(i> 100):
        continue
    print(foo,trigrams[foo])
f.close()

abgarus-acbarus-bio-4 emperor of constantinople  4. The contemporary of Trajan, who sent presents to that emperor when he invaded the east, and subsequently waited upon him and became his ally. ( D. C. 68.18. 21 .)  

alexander-bio-20 emperor of constantinople emperor of  CONSTANTINOPLE , was the third son of the emperor Basilius and Eudocia.

alexis-i-bio-1 emperor of constantinople  or ALE'XIUS I. COMNE'NUS ( Ἄλεξις  , or  Ἀλέξιος Κομνηνός ), emperor of Constantinople, was most probably born in  A. D. 1048 .

alexis-ii-bio-1 emperor of constantinople  or ALE'XIUS II. COMNE'NUS ( Ἄλεξις  or  Ἀλέξιος Κομνηνός ), emperor of Constantinople, the son of the emperor Manuel Comnenus, was born in  1167 , according to Nicetas. In  1179 , he married Agnes or Anna, the daughter of king Louis VII. of France, and succeeded his father in  1180 , under the guardianship of his mother Maria, the daughter of Raymond, prince of Antioch.

anastasius-i-bio-1 emperor of constantinople emperor of  CONSTANTI

julianus-flavius-claudius-bio-1 emperor of constantinople  surnamed  APOSTATA , "the Apostate," Roman emperor,  A. D. 361 - 363 , was born at Constantinople on the 17th of November,  A. D. 331  (332?).

justinianus-i-flavius-anicius-bio-1 emperor of constantinople  surnamed MAGNUS, or  THE GREAT , emperor of  CONSTANTINOPLE  and  ROME  from  A. D. 527  to  565 .

justinianus-ii-bio-1 emperor of constantinople  surnamed RHINOTME'TUS (he whose nose is cut off), emperor of the East ( A. D. 685 - 695  and  704 - 711 ), succeeded his father Constantine IV. Pogonatus, in the month of September,  A. D. 685 , at the age of sixteen.

justinus-i-bio-1 emperor of constantinople  I., or the elder, emperor of the East from  A. D. 518 - 527 , was of barbarian, probably Gothic extraction.

justinus-ii-bio-1 emperor of constantinople  the younger, emperor of the East, from  A. D. 565 - 578 , and nephew of the great Justinian. (See the genealogical table prefixed to the life of Justinian I.)

leo-i-fla

valens-bio-3 emperor of constantinople  emperor of the East  A. D. 364 - 378 , the brother of Valentinian [ VALENTINIANUS I. ], was born about  A. D. 328 .

zenon-bio-6 emperor of constantinople or ZENO, emperor of the East,  A.D. 474 - 491 , was descended from a noble Isaurian family.

zoe-empressname-bio-1 emperor of constantinople the name of several empresses of Constantinople, of whom the following were the most important :--    

done 286743


In [75]:
'calo-joannes' in singnameindex1

True

In [12]:
firstparatexts['thestius-bio-1']

' (  θέστιος  ), a son of ares and demonice or androdice, and, according to others, a son of agenor and a grandson of pleuron, the king of aetolia.\n'